## Setup

In [1]:
from openai import OpenAI
import backoff
from dotenv import load_dotenv
import os
from fastapi import HTTPException

In [2]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Checkers

In [15]:
response = client.beta.assistants.list()
print(response)

SyncCursorPage[Assistant](data=[], object='list', first_id=None, last_id=None, has_more=False)


In [4]:
client.beta.assistants.delete(response.data[0].id)

IndexError: list index out of range

In [31]:
response = client.beta.vector_stores.list()
print(response.data)

[VectorStore(id='vs_HUKS7D5SN8RXlLn73N0ORdTp', created_at=1714130154, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1714130159, metadata={}, name='Party Positions', object='vector_store', status='completed', usage_bytes=169121, expires_after=None, expires_at=None)]


In [37]:
response.data[0].file_counts.completed

1

In [30]:
client.beta.vector_stores.delete('vs_oWV03ZF99GGJJGnEM13hTyMh')

VectorStoreDeleted(id='vs_oWV03ZF99GGJJGnEM13hTyMh', deleted=True, object='vector_store.deleted')

## As in openai_assistant.py

In [3]:
# Helper function to create an assistant, if not found
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_assistant():
    try:
        # Making a call to create an assistant
        assistant = client.beta.assistants.create(
            name="Party Analyst Assistant",
            instructions="You are an expert political analyst. Use your knowledge base to answer questions about party positions.",
            model="gpt-4-turbo",
            tools=[{"type": "file_search"}],
        )
        print(f"Assistant created with ID: {assistant.id}")
        return assistant
    except Exception as e:
        print("Failed to create assistant:", e)
        raise HTTPException(status_code=500, detail=f"Failed to create assistant: {str(e)}")


In [4]:
# Helper function to get an assistant, returns the ID of the Political Analyst Assistant if found and creates a new one if not found
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def get_assistant():
    try:
        # Making a call to list assistantsassistant
        response = client.beta.assistants.list()  # Fetching all the assistants
        if response and response.data:
            for assistant in response.data:
                if assistant.name == "Party Analyst Assistant":
                    print("Party Analyst Assistant found.")
                    return assistant
            # If no assistant found in the loop, create a new one
            print("No Party Analyst Assistant found within, creating a new one...")
            return create_assistant()
        else:
            print("No assistants or data available or failed to fetch data, attempting to create a new assistant.")
            return create_assistant()
    except Exception as e:
        print("Failed to fetch assistants:", e)
        raise HTTPException(status_code=500, detail=f"Failed to fetch assistants: {str(e)}")


In [39]:
# Helper function to ensure the vector store is attached to the assistant
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def ensure_vector_store(assistant):
    try:
        # Check if the current assistant has the correct vector store attached
        vector_store_ids = assistant.tool_resources.file_search.vector_store_ids if assistant.tool_resources and assistant.tool_resources.file_search else []

        if vector_store_ids:
            # Fetch the vector store details
            vector_store = client.beta.vector_stores.retrieve(vector_store_id=vector_store_ids[0])
            if vector_store and vector_store.name == "Party Positions":
                print("Correct vector store already attached.")
                return vector_store
            else:
                print("Incorrect vector store attached, looking for correct vector store.")
        else:
            print("No vector store attached, looking for correct vector store.")

        # If the correct vector store is not attached, check if such a store exists
        all_stores = client.beta.vector_stores.list()
        party_positions_store = next((store for store in all_stores.data if store.name == "Party Positions"), None)

        if party_positions_store:
            vector_store_id = party_positions_store.id
            print(f"Vector store 'Party Positions' found with ID: {vector_store_id}")
        else:
            # Create a new vector store if not found
            print("Creating new vector store 'Party Positions'")
            vector_store = client.beta.vector_stores.create(name="Party Positions")
            vector_store_id = vector_store.id

            # Upload files to the new vector store
            file_paths = ["../data/party_positions.pdf"] # File path to the PDF file
            file_streams = [open(path, "rb") for path in file_paths]
            file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
                vector_store_id=vector_store_id, files=file_streams
            )
            print(f"Files uploaded to vector store: {file_batch.status}")

        # Attach the vector store to the assistant
        updated_assistant = client.beta.assistants.update(
            assistant_id=assistant.id,
            tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}}
        )
        print(f"Vector store '{vector_store_id}' attached to assistant.")
        return vector_store

    except Exception as e:
        print("Failed to ensure vector store:", e)
        raise HTTPException(status_code=500, detail=f"Failed to ensure vector store: {str(e)}")


In [44]:
# Main function to run the setup
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def assistant_setup():
    try:
        # Step 1: Get or create the assistant
        assistant = await get_assistant()
        print(f"Assistant retrieved or created with ID: {assistant.id}")

        # Step 2: Ensure the assistant has the correct vector store attached
        vector_store = await ensure_vector_store(assistant)
        print(f"Assistant is now correctly configured with vector store ID: {vector_store.id}")
        
        return assistant, vector_store

    except Exception as e:
        print("An error occurred in the main setup:", e)
        raise HTTPException(status_code=500, detail=str(e))

In [45]:
assistant, vector_store = await assistant_setup()

Party Analyst Assistant found.
Assistant retrieved or created with ID: asst_Wy1Tf8cA3bwO3wu5iSCxmF8W
Correct vector store already attached.
Assistant is now correctly configured with vector store ID: vs_HUKS7D5SN8RXlLn73N0ORdTp


## Not implemented yet

In [72]:
async def setup_and_query_conversation(assistant, vector_store_id, conversation_history):
    try:
        # Create a thread to handle the conversation
        thread = client.beta.threads.create(
            messages=[{
                "role": "user",
                "content": f"""
                            Your job is to analyze the given conversation, identify its topics and to give a thorough overview over the different party positions 
                            on all of the topics discussed in this conversation.
                            First, anlyze the conversation and identify the topics discussed. 
                            Then, look through the provided information in the vector store to find the party positions on the discussed topics.
                            Make sure to access and cite your knowledge base on party positions to provide the most accurate information. 
                            In the end, give a weighted, fair and balanced analysis of the different party positions on the discussed topics
                            and discuss what party could represent the users interests the best. This is the conversation: {conversation_history}"""
            }],
            tool_resources={
                "file_search": {
                    "vector_store_ids": [vector_store_id]
                }
            }
        )

        # Create and poll run until terminal state is reached
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant.id
        )

        # Retrieve all messages from the run
        messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

        if not messages:
            raise ValueError("No messages returned from the run.")

        # Assume first message contains the primary response
        message_content = messages[0].content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        return message_content.value, citations

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        # Handle specific exceptions or general exceptions and possibly retry or log errors



In [73]:
conversation_history = """

Socrates: We meet today to discuss a matter of great importance—climate issues, which I understand you have particular views on. Could you begin by stating your position?

Climate Skeptic: Certainly, Socrates. I believe the concern over climate change is overblown. The climate has always changed throughout history, and the current changes are part of a natural cycle, not primarily a result of human activities.

Socrates: A fascinating point. If I understand correctly, you are suggesting that because changes have occurred naturally in the past, the current changes must also be natural. Could you clarify whether you believe humans have had no role in these changes?

Climate Skeptic: Well, I wouldn't say no role at all, but the role is minimal. The main drivers are natural—volcanic activity, solar radiation, and variations in the Earth's orbit.

Socrates: Intriguing. If humans have played even a minimal role, might it be important to understand that role more deeply? After all, even small causes can have large effects.

Climate Skeptic: Perhaps, but the economic disruption from trying to change our energy use is too great based on such uncertain science.

Socrates: Let us examine this uncertainty. Is it not the case that many uncertainties in life lead us to take precautions—insurance against risks, for instance?

Climate Skeptic: That is true, but where do you draw the line? The costs of overhauling our industries and energy systems are enormous.

Socrates: A wise point. Costs should indeed be weighed. Yet, if the risk is great, such as irreversible damage to our environment, should we not consider the cost of inaction to be potentially even greater? What do you think is the wise course of action when the stakes are the survival of certain ecosystems or even human lives?

Climate Skeptic: The stakes are exaggerated. We adapt. Humans are resourceful.

Socrates: Adaptation is a remarkable human trait. But is foresight not also wise? If we can foresee potential damage and prevent it, is that not preferable to adapting to a degraded world?

Climate Skeptic: It might be, but the data must be irrefutable.

Socrates: Indeed, the quest for knowledge is never-ending. Let us then agree that seeking clearer, irrefutable data is crucial. Meanwhile, might it also be wise to prepare modestly for the chance that the current consensus on climate change might be correct? After all, the virtue of prudence lies in balancing between different kinds of risks.

Climate Skeptic: Perhaps a moderate approach might be reasonable.

Socrates: Then let us continue this discourse, always questioning, always seeking the truth, and always ready to act wisely upon it, balancing both skepticism and the precautionary principle. Thank you for engaging in this reflective conversation.

Climate Skeptic: Thank you, Socrates. It has given me much to think about.

                        """

message, citations = await setup_and_query_conversation(assistant, vector_store.id, conversation_history)

In [75]:
citations

['[0] party_positions.pdf',
 '[1] party_positions.pdf',
 '[2] party_positions.pdf',
 '[3] party_positions.pdf',
 '[4] party_positions.pdf',
 '[5] party_positions.pdf',
 '[6] party_positions.pdf',
 '[7] party_positions.pdf',
 '[8] party_positions.pdf']

In [55]:
# Use the create and poll SDK helper to create a run and poll the status of 
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

In Germany, views on immigration and environmental policy can vary across different political parties. Here's a general insight into where some of the major German parties stand on these issues:

1. **The Green Party (Die Grünen)**: This party prioritizes environmental issues strongly, advocating for sustainable practices and policies to combat climate change. However, the Greens are generally pro-immigration and support policies that provide protection to refugees and facilitate the integration of immigrants.

2. **Alternative for Germany (AfD)**: This party aligns more with the view you've expressed. The AfD is known for its skeptical stance on immigration, arguing that Germany cannot handle high levels of immigration. They also address environmental issues but are not as focused or progressive on them as the Greens. They tend to prioritize nationalist and conservative values, including more restrictive immigration policies.

3. **Christian Democratic Union (CDU)**: Traditionally, th

In [61]:
message_content.value

"In Germany, views on immigration and environmental policy can vary across different political parties. Here's a general insight into where some of the major German parties stand on these issues:\n\n1. **The Green Party (Die Grünen)**: This party prioritizes environmental issues strongly, advocating for sustainable practices and policies to combat climate change. However, the Greens are generally pro-immigration and support policies that provide protection to refugees and facilitate the integration of immigrants.\n\n2. **Alternative for Germany (AfD)**: This party aligns more with the view you've expressed. The AfD is known for its skeptical stance on immigration, arguing that Germany cannot handle high levels of immigration. They also address environmental issues but are not as focused or progressive on them as the Greens. They tend to prioritize nationalist and conservative values, including more restrictive immigration policies.\n\n3. **Christian Democratic Union (CDU)**: Traditiona